# Create MBTiles for Soils-revealed platform

Check this [notebook](https://github.com/Vizzuality/sci_team_data_bank/blob/master/Encyclopedia/map_tile_processing/MBTiles_from_Carto_data.ipynb) for further information on the creation of `MBTiles`.

## Table of Contents
### [Python libraries](#libraries)
### [Utils](#utils)
- **[df_from_carto](#df_from_carto)**
- **[long_lasting_SQL_queries](#long_lasting_SQL_queries)**
- **[create_mbtiles](#create_mbtiles)**

### [Read data from different sources ](#read_data)
- **[Biomes](#biomes)**
- **[World Database on Protected Areas](#protected_areas)**
- **[River basins](#river_basins)**
- **[Political boundaries](#political_boundaries)**

### [Create `MBTiles`](#create_mbtiles_2)
### [Show `MBTiles` in our localhost](#show_mbtiles)

<a id='libraries'></a>
### Python libraries

In [1]:
import os
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.wkb 
from shapely.ops import cascaded_union
from carto.auth import APIKeyAuthClient
from carto.sql import BatchSQLClient
from carto.sql import SQLClient
from tqdm import tqdm
import getpass
import subprocess
import time
import LMIPy

<a id='utils'></a>
### Utils
<a id='df_from_carto'></a>
**df_from_carto**

In [2]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

<a id='long_lasting_SQL_queries'></a>
**long_lasting_SQL_queries**

In [3]:
def long_lasting_SQL_queries(account, query, api_key):
    # For long lasting SQL queries we use the batch SQL API.
    table_name = 'job_result'
    
    base_url = f'https://{account}.carto.com/'
    auth_client = APIKeyAuthClient(api_key=api_key, base_url=base_url)
    
    sql_query =(f'SELECT * INTO {table_name} FROM ({query}) as job')
    
    LIST_OF_SQL_QUERIES = [sql_query]
    
    batchSQLClient = BatchSQLClient(auth_client)
    createJob = batchSQLClient.create(LIST_OF_SQL_QUERIES)
    
    # Check the status of a job with the job_id every 10 s
    readJob = batchSQLClient.read(createJob['job_id'])
    
    timeout = time.time() + 60*60  # 1 hour from now
    while readJob.get('status') != 'done':
        time.sleep(10)
        print(readJob.get('status'))
        if readJob.get('status') == 'failed':
            print('Job failed.')
            break
        if time.time() > timeout:
            readJob = batchSQLClient.read(createJob['job_id'])
            # Cancel a job given its job_id
            if readJob.get('status') != 'donne':
                cancelJob = batchSQLClient.cancel(createJob['job_id'])     
                print('Job cancelled after 1 hour running')
                break
            
        readJob = batchSQLClient.read(createJob['job_id'])
       
    # Read the table
    sql = SQLClient(auth_client)
    data = sql.send("select * from "+table_name)
    
    # Drop the table
    sql = SQLClient(auth_client)
    sql.send("DROP TABLE "+table_name)
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

**merge_geometries**

In [3]:
def merge_geometries(df, column_name):
    df_new = pd.DataFrame(columns=list(df.columns))
    geom = []
    for value in tqdm(df[column_name].unique()):
        df_tmp = df[df[column_name] == value].iloc[:1]
        geom.append(cascaded_union(list(df[df[column_name] == value].geometry)))
        
        df_new = pd.concat([df_new, df_tmp])
        
    df_new.reset_index(inplace=True)
    df_new.drop(columns='index', inplace=True)
    df_new.drop(columns='geometry', inplace=True)
    df_new['geometry'] = geom

    return gpd.GeoDataFrame(df_new)

<a id='create_mbtiles'></a>
**create_mbtiles**

In [4]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

<a id='read_data'></a>
### Read data from different sources 

<a id='biomes'></a>
#### **[Biomes](https://resourcewatch.org/data/explore/bio042-Ecoregion-by-Biome)**

In [ ]:
ds = LMIPy.Dataset('ed1544bb-a092-424e-88c2-8d548f4ef94a')
ds

In [ ]:
biomes = gpd.read_file('../data/mbtiles/bio_042_ecoregions_by_biome_1_14/bio_042_ecoregions_by_biome_1_14.shp')
biomes.drop(columns='cartodb_id', inplace=True)
biomes.head()

**Merge geometries**

In [ ]:
biomes_0 = merge_geometries(biomes, 'biome_name')

In [ ]:
a = np.empty(len(biomes_0))
a[:] = np.nan
biomes_0['eco_name'] = a
biomes_0['realm'] = a
biomes_0['eco_biome_'] = a
biomes_0['eco_id'] = a

Append `GeoDataFrames`

In [ ]:
biomes_0['level'] = 0
biomes['level'] = 1

In [ ]:
data = biomes_0.append(biomes, sort=False)
data = data[['biome_name', 'biome_num', 'eco_name', 'realm', 'eco_biome_', 'eco_id',
       'level', 'geometry']]
data

**Add id**

In [ ]:
data['id'] = np.arange(len(data))

In [ ]:
data.head()

**Add parent ID**

In [30]:
biomes = gpd.read_file('../data/mbtiles/ecoregions_by_biome/ecoregions_by_biome.shp')

biomes = pd.merge(biomes, biomes[biomes['level']==0][['biome_name', 'id']], on='biome_name', how='left').rename(columns={'id_x': 'id', 'id_y': 'id_0'})
columns = list(biomes.columns)
columns.insert(len(columns)-3, columns.pop(len(columns)-1))
data = biomes[columns]
data

,biome_name,biome_num,eco_name,realm,eco_biome_,eco_id,level,id,id_0,bbox,geometry
0,Tropical & Subtropical Moist Broadleaf Forests,1.0,None,None,None,NaN,0,0,0,"[-179.999419637, -34.12811629099991, 179.99923...","MULTIPOLYGON (((39.63333 -10.09917, 39.63500 -..."
1,Temperate Broadleaf & Mixed Forests,4.0,None,None,None,NaN,0,1,1,"[-176.8494566669999, -55.97751224299998, 178.5...","MULTIPOLYGON (((-67.24305 -55.93486, -67.23734..."
2,Tropical & Subtropical Dry Broadleaf Forests,2.0,None,None,None,NaN,0,2,2,"[-160.21405066399993, -22.315159577999964, 179...","MULTIPOLYGON (((166.00638 -21.98935, 165.99634..."
3,Tropical & Subtropical Coniferous Forests,3.0,None,None,None,NaN,0,3,3,"[-111.19810459799999, -2.3255536319999237, 121...","MULTIPOLYGON (((-75.05347 20.68018, -75.05943 ..."
4,Temperate Conifer Forests,5.0,None,None,None,NaN,0,4,4,"[-152.94373648999994, 24.98491065800016, 145.8...","MULTIPOLYGON (((91.83731 27.42280, 91.81181 27..."
...,...,...,...,...,...,...,...,...,...,...,...
2680,N/A,11.0,Rock and Ice,N/A,N/A,0.0,1,2680,13,"[-159.33110340899998, -86.41899574699994, -159...","POLYGON ((-159.14277 -86.41071, -159.07742 -86..."
2681,N/A,11.0,Rock and Ice,N/A,N/A,0.0,1,2681,13,"[-165.64174892799994, -86.48082001799992, -165...","POLYGON ((-165.41384 -86.47713, -165.42556 -86..."
2682,N/A,11.0,Rock and Ice,N/A,N/A,0.0,1,2682,13,"[-164.40798912399993, -86.47827124799988, -164...","POLYGON ((-164.31972 -86.47755, -164.36305 -86..."
2683,N/A,11.0,Rock and Ice,N/A,N/A,0.0,1,2683,13,"[-165.32888873799993, -86.57706592999995, -165...","POLYGON ((-165.20552 -86.57379, -165.24149 -86..."


Save `GeoDataFrame` as `GeoJSON`

In [31]:
data.to_file("../data/mbtiles/ecoregions_by_biome.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [32]:
data.to_file("../data/mbtiles/ecoregions_by_biome/ecoregions_by_biome.shp")

<a id='protected_areas'></a>
#### **[World Database on Protected Areas](https://resourcewatch.org/data/explore/bio007-World-Database-on-Protected-Areas_replacement)**

In [ ]:
ds = LMIPy.Dataset('2442891a-157a-40e6-9092-ee596e6d30ba')
ds

In [ ]:
areas = gpd.read_file('../data/mbtiles/wdpa_protected_areas/wdpa_protected_areas.shp')
areas.drop(columns='cartodb_id', inplace=True)
areas.head()

In [ ]:
areas_filter = areas[(areas['marine'] == '0') & (areas['iucn_cat'] != 'Not Assigned') & (areas['iucn_cat'] != 'Not Applicable') & (areas['iucn_cat'] != 'Not Reported')]

In [ ]:
areas['iucn_cat'].unique()

In [ ]:
account = "wri-01"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
query = 'SELECT distinct(iucn_cat) FROM wdpa_protected_areas'
df = df_from_carto(account, query)
iucn_cats = list(df['iucn_cat'])

In [ ]:
query = 'SELECT * FROM wdpa_protected_areas LIMIT 1'
df = df_from_carto(account, query)
coulmns = list(df.columns)

In [ ]:
data = gpd.GeoDataFrame(columns=coulmns)

for iucn_cat in iucn_cats:
    query = "SELECT * FROM wdpa_protected_areas WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III'"

    df = long_lasting_SQL_queries(account, query, api_key)
    
    data = pd.concat([data, df], ignore_index=True)

In [ ]:
data.head()

<a id='river_basins'></a>
#### **River basins** ([source](http://www.fao.org/nr/water/aquamaps/))
**Major hydrological basins**

In [ ]:
major = gpd.read_file('../data/mbtiles/Major_hydrological_basins/major_hydrobasins.shp')
major.columns = map(str.lower, major.columns)
major.head()

**Merge geometries**

In [ ]:
major = merge_geometries(major, 'maj_bas')

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
major.to_file("../data/mbtiles/major_hydrological_basins.json", driver="GeoJSON")

**Minor hydrological basins**

In [ ]:
account = "wri-rw"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
query = "SELECT maj_bas, maj_name, maj_area, sub_bas, sub_name, sub_area, the_geom FROM hydrobasins_fao_fiona_merged_v01"

minor = long_lasting_SQL_queries(account, query, api_key)

In [ ]:
minor.head()

**Merge geometries**

In [ ]:
minor = merge_geometries(minor, 'sub_bas')

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
minor.to_file("../data/mbtiles/minor_hydrological_basins.json", driver="GeoJSON")

Append `GeoDataFrames`

In [ ]:
major['level'] = 0
minor['level'] = 1

In [ ]:
data = major.append(minor, sort=False)
data = data[['maj_bas', 'maj_name', 'maj_area', 'sub_bas', 'sub_name', 'sub_area',
       'level', 'geometry']]
data

**Add id**

In [ ]:
data['id'] = np.arange(len(data))

**Add parent ID**

In [33]:
data = gpd.read_file('../data/mbtiles/hydrological_basins/hydrological_basins.shp')

In [35]:
data = pd.merge(data, data[data['level']==0][['maj_name', 'id']], on='maj_name', how='left').rename(columns={'id_x': 'id', 'id_y': 'id_0'})
columns = list(data.columns)
columns.insert(len(columns)-3, columns.pop(len(columns)-1))
data = data[columns]
data

,maj_bas,maj_name,maj_area,sub_bas,sub_name,sub_area,level,id,id_0,bbox,geometry
0,4050,Arctic Ocean Islands,2166086,None,None,None,0,0,0,"[-73.2624999999999, 59.779166666666754, 69.070...","MULTIPOLYGON (((-43.596 59.813, -43.612 59.813..."
1,1012,Northwest Territories,3044095,None,None,None,0,1,1,"[-136.27499999999998, 53.10416666666675, -61.0...","MULTIPOLYGON (((-111.392 54.171, -111.387 54.1..."
2,5064,"Siberia, North Coast",1200168,None,None,None,0,2,2,"[79.0541666666669, 65.77916666666675, 129.6083...","MULTIPOLYGON (((129.083 72.479, 129.075 72.479..."
3,5068,"Siberia, West Coast",3052334,None,None,None,0,3,3,"[-180.0, 43.35833333333341, 180.00000000000028...","MULTIPOLYGON (((146.000 43.358, 145.988 43.358..."
4,5066,Kara Sea Coast,696301,None,None,None,0,4,4,"[64.94583333333352, 62.43750000000009, 86.1541...","MULTIPOLYGON (((78.825 67.571, 78.800 67.571, ..."
...,...,...,...,...,...,...,...,...,...,...,...
5096,2015,Caribbean,232942,15044,Este,7864,1,5096,111,"[-69.69166666666733, 18.191666666666045, -68.3...","MULTIPOLYGON (((-69.004 18.379, -69.004 18.371..."
5097,2015,Caribbean,232942,15049,Atlantica,4796,1,5097,111,"[-71.12916666666726, 19.162499999999365, -69.1...","MULTIPOLYGON (((-69.271 19.171, -69.271 19.162..."
5098,2015,Caribbean,232942,15020,Estère,815,1,5098,111,"[-72.77499999990994, 19.03333333333303, -72.26...","POLYGON ((-72.742 19.321, -72.733 19.321, -72...."
5099,2015,Caribbean,232942,15053,Limonade / Ouanaminthe,1500,1,5099,111,"[-72.12916666666725, 19.33333333333304, -71.55...","POLYGON ((-72.121 19.742, -72.121 19.737, -72...."


Save `GeoDataFrame` as `GeoJSON`

In [37]:
data.to_file("../data/mbtiles/hydrological_basins.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [38]:
data.to_file("../data/mbtiles/hydrological_basins/hydrological_basins.shp")

<a id='political_boundaries'></a>
#### **Political boundaries ([source](https://gadm.org/data.html))**

**gadm36 political boundaries**

In [ ]:
account = "wri-01"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
account = "wri-01"

query = "SELECT name_0, name_1, name_2, area, size, level, gid_0, gid_1, gid_2, the_geom FROM gadm36_political_boundaries"

data = long_lasting_SQL_queries(account, query, api_key)
data.head()

In [ ]:
data.plot()

In [ ]:
gadm36 = gpd.read_file('../data/mbtiles/gadm36_political_boundaries/gadm36_political_boundaries.shp')
gadm36.drop(columns='cartodb_id', inplace=True)
gadm36.head()

In [ ]:
gadm36 = gadm36[gadm36['level'].isin([0,1])]

**Add id**

In [ ]:
gadm36['id'] = np.arange(len(gadm36))

**Add parent ID**

In [39]:
data = gpd.read_file('../data/mbtiles/political_boundaries/political_boundaries.shp')

In [41]:
data = pd.merge(data, data[data['level']==0][['gid_0', 'id']], on='gid_0', how='left').rename(columns={'id_x': 'id', 'id_y': 'id_0'})
columns = list(data.columns)
columns.insert(len(columns)-3, columns.pop(len(columns)-1))
data = data[columns]
data

,name_1,area,gid_0,gid_1,gid_2,level,name_0,name_2,size,simple_nam,simple_n_1,simple_n_2,name_es_mx,name_zh,name_fr,id,id_0,bbox,geometry
0,Vestfirðir,0.000000e+00,ISL,ISL.7_1,None,1,Iceland,None,medium,Iceland,Vestfirdir,None,Islandia,??,Islande,0,919,"[-25.0349, 64.9637, -20.9778, 66.7021]","MULTIPOLYGON (((-20.97950 64.96370, -21.05620 ..."
1,Saint Ann,1.222664e+05,JAM,JAM.7_1,None,1,Jamaica,None,small,Jamaica,Saint Ann,None,Jamaica,???,Jamaïque,1,1517,"[-77.4874, 18.183, -76.9981, 18.4801]","MULTIPOLYGON (((-77.48730 18.20240, -77.48740 ..."
2,Mascara,5.503186e+05,DZA,DZA.27_1,None,1,Algeria,None,medium,Algeria,Mascara,None,Argelia,?????,Algérie,2,171,"[-0.5208, 35.0109, 0.9031, 35.7845]","POLYGON ((0.28860 35.05840, 0.28830 35.05760, ..."
3,Saint James,5.932997e+04,JAM,JAM.10_1,None,1,Jamaica,None,small,Jamaica,Saint James,None,Jamaica,???,Jamaïque,3,1517,"[-77.9958, 18.2135, -77.7342, 18.5251]","MULTIPOLYGON (((-77.73420 18.51070, -77.75460 ..."
4,None,0.000000e+00,WLF,None,None,0,Wallis and Futuna,None,very small,Wallis and Futuna,None,None,Wallis y Futuna,None,Wallis et Futuna,4,4,"[-178.1825, -14.3691, -176.1242, -13.1828]","MULTIPOLYGON (((-176.12610 -13.28720, -176.125..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3859,None,5.267234e+01,VAT,None,None,0,Vatican City,None,very small,Vatican City,None,None,Ciudad del Vaticano,???,Saint-Siège (État de la Cité du Vatican),3859,3859,"[12.4456, 41.9001, 12.4585, 41.9076]","POLYGON ((12.45560 41.90760, 12.45580 41.90640..."
3860,None,0.000000e+00,XCL,None,None,0,Clipperton Island,None,very small,Clipperton Island,None,None,None,None,None,3860,3860,"[-109.2349, 10.2851, -109.1982, 10.3204]","POLYGON ((-109.22540 10.32040, -109.22540 10.3..."
3861,None,1.006455e+04,MSR,None,None,0,Montserrat,None,very small,Montserrat,None,None,Montserrat,None,Montserrat,3861,3861,"[-62.2418, 16.6749, -62.1446, 16.8243]","POLYGON ((-62.15010 16.71210, -62.15010 16.711..."
3862,None,3.621598e+04,VIR,None,None,0,"Virgin Islands, U.S.",None,very small,Virgin Islands U.S.,None,None,Islas Virgenes Americanas,None,Îles Vierges des États-Unis,3862,3862,"[-65.0865, 17.6729, -64.5649, 18.4157]","MULTIPOLYGON (((-64.82210 17.68650, -64.82210 ..."


Save `GeoDataFrame` as `GeoJSON`

In [42]:
data.to_file("../data/mbtiles/political_boundaries.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [43]:
data.to_file("../data/mbtiles/political_boundaries/political_boundaries.shp")

**Disputed boundaries 2018**

In [ ]:
account = "wri-01"

query = "SELECT gid_0, name_0, name, note, the_geom FROM disputed_boundaries_2018 WHERE gid_0 in ('PAK', 'IND', 'CHN') AND name in ('Indian claim', 'Pakistani claim', 'Chinese claim')"

data = df_from_carto(account, query)
data

#### Major Physical Features ([source](file:///Users/ikersanchez/Vizzuality/GitHub/sci_team_data_bank/Projects/soils-revealed/data/mbtiles/ne_10m_geography_regions_elevation_points/ne_10m_geography_regions_elevation_points.README.html))
**Physical areas**

In [24]:
polys = gpd.read_file('../data/mbtiles/ne_10m_geography_regions_polys/ne_10m_geography_regions_polys.shp')
polys = polys[~polys.featurecla.isin(['Continent','Island group', 'Dragons-be-here', 'Lake'])]
polys.head()

,featurecla,name,namealt,region,subregion,min_label,max_label,scalerank,label,wikidataid,...,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,wdid_score,ne_id,geometry
0,Island,Pentecost,None,Oceania,Melanesia,6.7,12.0,7,Pentecost I.,Q960808,...,Pentecost,Pentecostes,Пентекост,Pentecost Island,None,None,彭特科斯特岛,4,1159102327,"POLYGON ((168.22657 -15.98968, 168.20411 -15.9..."
1,Island,Anatom,None,Oceania,Melanesia,6.7,12.0,7,Anatom,Q528884,...,Anatom,Aneytium,Анейтьюм,None,Anatom,None,阿納托姆島,4,1159102329,"POLYGON ((169.83790 -20.24737, 169.82997 -20.2..."
2,Island,Gaua,None,Oceania,Melanesia,6.7,12.0,7,Gaua,Q583383,...,Gaua,Gaua,Гауа,None,None,None,加瓦島,4,1159102331,"POLYGON ((167.52491 -14.14214, 167.58619 -14.1..."
3,Island,Utupua,None,Oceania,Melanesia,6.7,12.0,7,Utupua I.,Q2081472,...,None,None,Утупуа,Utupua,None,None,None,4,1159102333,"POLYGON ((166.52223 -11.27068, 166.52711 -11.2..."
4,Island,Vanikolo,None,Oceania,Melanesia,6.7,12.0,7,Vanikolo,Q1536424,...,Vanikoro,None,Ваникоро,Vanikoro,None,None,None,4,1159102335,"POLYGON ((166.93263 -11.66021, 166.91822 -11.6..."


**Merge geometries**

In [25]:
polys_0 = merge_geometries(polys, 'featurecla')

100%|██████████| 19/19 [00:00<00:00, 40.08it/s]


In [26]:
a = np.empty(len(polys_0))
a[:] = np.nan
for column in list(polys_0.columns)[1:-1]:
    polys_0[column] = a

Append `GeoDataFrames`

In [27]:
polys_0['level'] = 0
polys['level'] = 1

data = polys_0.append(polys, sort=False)

data = data[['featurecla', 'name', 'region', 'subregion', 'ne_id', 'level', 'geometry']]
data

,featurecla,name,region,subregion,ne_id,level,geometry
0,Island,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-159.042 -79.708, -159.542 -79..."
1,Coast,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-158.042 -84.792, -163.176 -84..."
2,Range/mtn,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-154.926 -85.316, -150.290 -85..."
3,Pen/cape,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-75.336 -46.936, -75.574 -46.9..."
4,Desert,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((140.931 -31.089, 140.805 -31.0..."
...,...,...,...,...,...,...,...
1038,Pen/cape,Coromandel,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((175.336 -36.411, 175.465 -36.418, 17..."
1039,Plateau,N.I. Volcanic Plateau,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((175.721 -38.867, 175.980 -38.956, 17..."
1040,Coast,Marlborough Sounds,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((173.725 -40.798, 173.856 -40.708, 17..."
1041,Pen/cape,Otago Pen.,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((170.654 -45.716, 170.740 -45.721, 17..."


**Add id**

In [28]:
data['id'] = np.arange(len(data))

**Add parent ID**

In [49]:
data = gpd.read_file("../data/mbtiles/ne_10m_geography_regions/ne_10m_geography_regions.shp")
data['name'] = data['name'].str.title() 

,featurecla,name,region,subregion,ne_id,level,id,bbox,geometry
0,Island,None,None,None,NaN,0,0,"[-180.00000000000003, -80.76580359797957, 179....","MULTIPOLYGON (((-159.042 -79.708, -159.542 -79..."
1,Coast,None,None,None,NaN,0,1,"[-165.74999129445237, -84.79156043391708, 174....","MULTIPOLYGON (((-158.042 -84.792, -163.176 -84..."
2,Range/mtn,None,None,None,NaN,0,2,"[-179.9999893871038, -87.64739539485454, 180.0...","MULTIPOLYGON (((-154.926 -85.316, -150.290 -85..."
3,Pen/cape,None,None,None,NaN,0,3,"[-179.13182723195243, -46.96233489375588, 176....","MULTIPOLYGON (((-75.336 -46.936, -75.574 -46.9..."
4,Desert,None,None,None,NaN,0,4,"[-119.2857559886663, -31.09148528438088, 143.1...","MULTIPOLYGON (((140.931 -31.089, 140.805 -31.0..."
...,...,...,...,...,...,...,...,...,...
890,Pen/cape,Coromandel,Oceania,New Zealand,1.159105e+09,1,890,"[175.3074647566757, -37.360183392074646, 176.0...","POLYGON ((175.336 -36.411, 175.465 -36.418, 17..."
891,Plateau,N.I. Volcanic Plateau,Oceania,New Zealand,1.159105e+09,1,891,"[175.29279141380664, -39.63732757465391, 176.0...","POLYGON ((175.721 -38.867, 175.980 -38.956, 17..."
892,Coast,Marlborough Sounds,Oceania,New Zealand,1.159105e+09,1,892,"[173.464086401038, -41.36536155803435, 174.431...","POLYGON ((173.725 -40.798, 173.856 -40.708, 17..."
893,Pen/cape,Otago Pen.,Oceania,New Zealand,1.159105e+09,1,893,"[170.48449558627487, -45.95479654513372, 170.8...","POLYGON ((170.654 -45.716, 170.740 -45.721, 17..."


In [50]:
data = pd.merge(data, data[data['level']==0][['featurecla', 'id']], on='featurecla', how='left').rename(columns={'id_x': 'id', 'id_y': 'id_0'})
columns = list(data.columns)
columns.insert(len(columns)-3, columns.pop(len(columns)-1))
data = data[columns]
data

,featurecla,name,region,subregion,ne_id,level,id,id_0,bbox,geometry
0,Island,None,None,None,NaN,0,0,0,"[-180.00000000000003, -80.76580359797957, 179....","MULTIPOLYGON (((-159.042 -79.708, -159.542 -79..."
1,Coast,None,None,None,NaN,0,1,1,"[-165.74999129445237, -84.79156043391708, 174....","MULTIPOLYGON (((-158.042 -84.792, -163.176 -84..."
2,Range/mtn,None,None,None,NaN,0,2,2,"[-179.9999893871038, -87.64739539485454, 180.0...","MULTIPOLYGON (((-154.926 -85.316, -150.290 -85..."
3,Pen/cape,None,None,None,NaN,0,3,3,"[-179.13182723195243, -46.96233489375588, 176....","MULTIPOLYGON (((-75.336 -46.936, -75.574 -46.9..."
4,Desert,None,None,None,NaN,0,4,4,"[-119.2857559886663, -31.09148528438088, 143.1...","MULTIPOLYGON (((140.931 -31.089, 140.805 -31.0..."
...,...,...,...,...,...,...,...,...,...,...
890,Pen/cape,Coromandel,Oceania,New Zealand,1.159105e+09,1,890,3,"[175.3074647566757, -37.360183392074646, 176.0...","POLYGON ((175.336 -36.411, 175.465 -36.418, 17..."
891,Plateau,N.I. Volcanic Plateau,Oceania,New Zealand,1.159105e+09,1,891,5,"[175.29279141380664, -39.63732757465391, 176.0...","POLYGON ((175.721 -38.867, 175.980 -38.956, 17..."
892,Coast,Marlborough Sounds,Oceania,New Zealand,1.159105e+09,1,892,1,"[173.464086401038, -41.36536155803435, 174.431...","POLYGON ((173.725 -40.798, 173.856 -40.708, 17..."
893,Pen/cape,Otago Pen.,Oceania,New Zealand,1.159105e+09,1,893,3,"[170.48449558627487, -45.95479654513372, 170.8...","POLYGON ((170.654 -45.716, 170.740 -45.721, 17..."


Save `GeoDataFrame` as `GeoJSON`

In [51]:
data.to_file("../data/mbtiles/ne_10m_geography_regions.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [52]:
data.to_file("../data/mbtiles/ne_10m_geography_regions/ne_10m_geography_regions.shp")

### Add bounding box

In [31]:
datasets = ['ne_10m_geography_regions', 'ecoregions_by_biome', 'political_boundaries', 'hydrological_basins']
datasets = ['ne_10m_geography_regions']
for dataset in datasets:
    gdf = gpd.read_file(f"../data/mbtiles/{dataset}/{dataset}.shp")
    gdf['bbox'] = gdf['geometry'].apply(lambda x: str(list(x.bounds)))
    gdf.to_file(f"../data/mbtiles/{dataset}.json", driver="GeoJSON")
    gdf.to_file(f"../data/mbtiles/{dataset}/{dataset}.shp")

<a id='create_mbtiles_2'></a>
### Create `MBTiles`

In [54]:
layers = {'Hydrological basins': 'hydrological_basins.json',
         'Ecoregions': 'ecoregions_by_biome.json',
         'Political boundaries': 'political_boundaries.json',
         'Physical geography regions': 'ne_10m_geography_regions.json'}

for layer_name, file in layers.items():
    print(layer_name)
    source_path = "../data/mbtiles/"+file
    dest_path = "../data/mbtiles/"+file.split('.')[0]+".mbtiles"
    create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Hydrological basins
Processing: tippecanoe -o ../data/mbtiles/hydrological_basins.mbtiles -l Hydrological basins -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/hydrological_basins.json
Task created
Finished processing
Ecoregions
Processing: tippecanoe -o ../data/mbtiles/ecoregions_by_biome.mbtiles -l Ecoregions -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/ecoregions_by_biome.json
Task created
Finished processing
Political boundaries
Processing: tippecanoe -o ../data/mbtiles/political_boundaries.mbtiles -l Political boundaries -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/political_boundaries.json
Task created
Finished processing
Physical geography regions
Processing: tippecanoe -o ../data/mbtiles/ne_10m_geography_regions.mbtiles -l Physical geography regions -zg --drop-densest-as-needed --extend-zooms-if-still-dropping -

<a id='show_mbtiles'></a>
### Show `MBTiles` in our localhost

In [18]:
!mbview --port 9000 ../data/mbtiles/ne_10m_geography_regions.mbtiles

Listening on http://localhost:9000
^C


<a id='serve_mbtiles'></a>
### 7. Serve `MBTiles` in our localhost

In [ ]:
!docker run --rm -p 8080:8000 -v /Users/ikersanchez/Vizzuality/GitHub/sci_team_data_bank/Projects/soils-revealed/data/mbtiles:/tilesets  consbio/mbtileserver

Paste in your browser the following:

`http://localhost:8080/services/hydrological_basins`

And to see the tiles on a map:

`http://localhost:8080/services/hydrological_basins/map`